In [1]:
from pyspark import SparkContext
import numpy as np
from sklearn.neighbors import NearestNeighbors, BallTree, KDTree

In [2]:
def find_partial_connected_components(data,cutoff=15.0):

    import networkx as nx
    import numpy as np
    from sklearn.neighbors import BallTree
    
    tree = BallTree(data, leaf_size=40)
    edges = tree.query_radius(data, cutoff)
    edge_list=[list(zip(np.repeat(idx, len(dest_list)), \
            dest_list)) for idx, dest_list in enumerate(edges)]

    edge_list_flat = np.array([list(item) \
            for sublist in edge_list for item in sublist])
    res = edge_list_flat
    res_tree = edge_list_flat[edge_list_flat[:,0]<edge_list_flat[:,1], :]

    graph =nx.from_edgelist(res_tree)

    # partial connected components

    connected_components = nx.connected_components(graph)
    for x in connected_components:
        yield [x]

In [3]:
def merge_spanning_trees(c1,c2):
    while len(c2)!=0:
        temp = c2[0]
        c2.pop(0)
        for i,item in enumerate(c1):
            if item.intersection(temp):
                temp.union(item)
                c1.pop(i)
        c1.append(temp)
        
    return c1

In [7]:
filename = '/Users/giorgos/repos/midas/leaflet_finder/132k_dataset/atom_pos_132K.npy'
coord_matrix = np.load(filename)  #points_local_np
matrix_size = coord_matrix.shape[0]
partition_length = matrix_size/10   # Fix this: task size

arranged_coord = list()
for i in range(0,matrix_size,partition_length):
    arranged_elem = coord_matrix[i:i+partition_length,:]
    arranged_coord.append(arranged_elem)

dist_Matrix = sc.parallelize(arranged_coord,len(arranged_coord))
connected_components = dist_Matrix.flatMap(find_partial_connected_components).reduce(merge_spanning_trees)
indices = [np.sort(list(g)) for g in connected_components]